# Twitchly Quick Start Guide


## Set Up
To start off with understanding Twitchly, we will first dive into the [Twith API](https://dev.twitch.tv/docs/api/).

We will learn about using your `clientID` to make calls to the Twitch API!

First, follow the quick set up guide on the link above to register your application.

For the sake of this demo
we can use the following values when registering:
- Name = APIDemo
- OAuth Redirect URL = http://localhost:5000/
- Category = Analytics Tool

Once you have created your application, you can click `manage` to find your `clientID` and `clientSecret`.

Next, we will use a wrapper python library called [python-twitch-client](https://python-twitch-client.readthedocs.io/en/latest/index.html).
That will tell you how to install the library as well as provide some very useful documentation.
For now, we will run through an example where we use the popular streamer [Ninja](https://en.wikipedia.org/wiki/Ninja_(streamer)
and get his user data to find out who he is following and who those followed channels are following.

In [5]:
from twitch import TwitchClient 
import pandas as pd


# Let's create the client by passing in your specific client id.
client = TwitchClient(client_id = "zjwe67emf2ri3ecyqvihgzhb1r3l4i")

# Most of the API is reliant on user-ids so this useful query can provide us the list of user-ids
# for the given usernames that we are all familiar with.
queried_users = client.users.translate_usernames_to_ids(['Ninja'])

# Now that we have the list of users, let's extract Ninja from the list.
user = queried_users[0]

# The user data is full of great information but for now, let's just verify that we have the username and id.
print(user.display_name, user.id)

Ninja 19571641


In [6]:
# Let's now query for the channels that ninja follows.
# For this example, let's just grab 5 channels.
# Check out the documentation to find out some other functionalities of this function.
followed_channels = client.users.get_follows(user.id, limit = 5)

# We filter out the result a little and add them into a list.
channels = []
for channel in followed_channels:
    channels.append(channel.channel)

# For the fun of it, let's throw these results into a Pandas Dataframe.
pd.DataFrame.from_records(channels)

,broadcaster_language,broadcaster_software,broadcaster_type,created_at,description,display_name,followers,game,id,language,...,partner,privacy_options_enabled,private_video,profile_banner,profile_banner_background_color,status,updated_at,url,video_banner,views
0,en,unknown_rtmp,partner,2016-07-04 00:22:25,Baseballer and Gamer and occasional DJ. @Trevm...,iamtrevormay,48380,Fortnite,128479231,en,...,True,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,None,Last stream from Minneapolis! !youtube !subgoa...,2018-10-01 05:54:33,https://www.twitch.tv/iamtrevormay,https://static-cdn.jtvnw.net/jtv_user_pictures...,432454
1,en,unknown_rtmp,partner,2013-07-21 04:09:19,Check out my YouTube channel: http://www.youtu...,WILDCAT,644771,Cities: Skylines,46386566,en,...,True,False,False,None,#ffffff,im bad,2018-10-01 05:52:18,https://www.twitch.tv/wildcat,https://static-cdn.jtvnw.net/jtv_user_pictures...,2833023
2,en,unknown_rtmp,partner,2018-04-13 22:09:57,The home for Beyond The Summit's unique and au...,btssmash,50216,Super Smash Bros. Melee,214062798,en,...,True,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,None,RERUN: aMSa vs n0ne - Melee Singles: Losers' R...,2018-10-01 05:52:19,https://www.twitch.tv/btssmash,https://static-cdn.jtvnw.net/jtv_user_pictures...,2161951
3,en,unknown_rtmp,affiliate,2009-07-01 13:06:10,Check out my stream for retro style games like...,Zexxios,952,Left 4 Dead,7054369,en,...,False,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,#002cbd,"Spooktober Day #8 - RE7 Wrap up, Left 4 Dead w...",2018-10-01 03:00:55,https://www.twitch.tv/zexxios,https://static-cdn.jtvnw.net/jtv_user_pictures...,9514
4,en,unknown_rtmp,partner,2012-05-11 22:56:04,"Hi, I'm Marcel, but you can call me BasicallyI...",BasicallyIDoWrk,258537,Fortnite,30446023,en,...,True,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,None,I'm really bad.,2018-10-01 05:53:42,https://www.twitch.tv/basicallyidowrk,https://static-cdn.jtvnw.net/jtv_user_pictures...,995283


In [8]:
# Let's write a quick function that will take in the list of channels,
# query up to k followers for each channel and then add them all into
# a list as output.
def multi_follow_query(channels, k):
    result = []
    for channel in channels:
        followed_channels = client.users.get_follows(channel.id, limit = k)
        for followed_channel in followed_channels:
            result.append(followed_channel.channel)
    return result

# Let's query for results from the list of channels that Ninja is following.
result = multi_follow_query(channels, 2)

pd.DataFrame.from_records(result)

,broadcaster_language,broadcaster_software,broadcaster_type,created_at,description,display_name,followers,game,id,language,...,partner,privacy_options_enabled,private_video,profile_banner,profile_banner_background_color,status,updated_at,url,video_banner,views
0,en,unknown_rtmp,partner,2018-05-18 18:47:42,Jordan Fisher,JordanFisher,15166,Fortnite,224145872,en,...,True,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,None,"Squads with Fear, Justin Ola and Train || Last...",2018-10-01 04:56:08,https://www.twitch.tv/jordanfisher,https://static-cdn.jtvnw.net/jtv_user_pictures...,101080
1,en,unknown_rtmp,partner,2015-03-24 22:02:30,I stream games and stuff,GubbaTV,101549,just chatting,86198803,en,...,True,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,None,Can we hit 1800 subs? !patreon for exclusive c...,2018-10-01 04:26:49,https://www.twitch.tv/gubbatv,https://static-cdn.jtvnw.net/jtv_user_pictures...,5149231
2,en,unknown_rtmp,partner,2017-02-12 11:29:55,Just a dude who loves making video games =D,Carto,12596,H1Z1,147595677,en,...,True,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,None,Sub Night! Do A !steam then !discord to join i...,2018-10-01 05:54:12,https://www.twitch.tv/carto,https://static-cdn.jtvnw.net/jtv_user_pictures...,206055
3,en,unknown_rtmp,affiliate,2013-08-28 02:36:08,I stream everyday from 2pm to 2am CST!,yoyokeepitup,13327,Fortnite,48225190,en,...,False,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,None,Pro Duo Scrims | !Giveaway | !YouTube | !Disco...,2018-10-01 05:54:38,https://www.twitch.tv/yoyokeepitup,https://static-cdn.jtvnw.net/jtv_user_pictures...,116205
4,en,unknown_rtmp,partner,2017-11-23 18:21:05,,TwitchPresents2,99653,Always On,182953678,en,...,True,False,False,None,None,Hosting Thunderbirds! POKÉMON RERUN FRIDAYS AN...,2018-10-01 05:53:00,https://www.twitch.tv/twitchpresents2,https://static-cdn.jtvnw.net/jtv_user_pictures...,9027581
5,en,unknown_rtmp,affiliate,2018-03-06 01:49:04,Variety streamer. I play retro games Mon/Wed a...,HarmonyGinger,303,Retro,202998830,en,...,False,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,None,"VATS says I have a 95% chance of hitting that,...",2018-09-30 02:23:51,https://www.twitch.tv/harmonyginger,https://static-cdn.jtvnw.net/jtv_user_pictures...,7410
6,en,unknown_rtmp,partner,2013-07-30 22:49:04,,Thiefs,114890,Fortnite,46865623,en,...,True,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,None,"FaZe Thiefs - $10,000 NerdOut Sabotage Tourney...",2018-10-01 05:54:10,https://www.twitch.tv/thiefs,https://static-cdn.jtvnw.net/jtv_user_pictures...,1229180
7,en,unknown_rtmp,partner,2015-02-03 07:50:36,Battle Royale Pro Player.,cloakzy,609977,Fortnite,81687332,en,...,True,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,None,scrims w/ tofue,2018-10-01 05:53:15,https://www.twitch.tv/cloakzy,https://static-cdn.jtvnw.net/jtv_user_pictures...,5679273
